<a href="https://colab.research.google.com/github/koheiKESE/ai-seminer/blob/main/06_detectron2%E3%81%AB%E3%82%88%E3%82%8B%E8%87%AA%E5%89%8D%E7%94%BB%E5%83%8F%E3%81%AE%E6%8E%A8%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. このノートブックを実行する前すること

1. ランタイムをハードウェアアクセラレータをGPUに指定
  - ランタイム -> ランタイムのタイプを変更 -> ハードウェアアクセラレータをGPUに設定


2. 推論させたい画像をGoogle Drive直下にコピー

3. このノートブックをGoogle Driveに保存 ( +コード＋テキストの右隣に表示されている「ドライブにコピー」ボタンをクリック

# 1. 初期設定

  1. Google Driveをマウント

  2. detectron2をインストール (**注意** : 90分/12時間ルールによりインストールしたものはリセットされます)

  3. (インストール完了後)colabの環境とdetectron2の環境間のバージョンのコンフリクトを解消するため、ランタイムを再起動 

In [ ]:
# Google Driveを /content/drive以下にマウント
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# detectron2
# 所要時間 30秒程度
!pip install pyyaml==5.1
!pip install 'git+https://github.com/facebookresearch/detectron2.git' --upgrade

# インストール完了後、ランタイムを再起動

## 表示内容
WARNING: The following packages were previously imported in this runtime:
  [certifi]
You must restart the runtime in order to use newly installed versions.


## 解決方法 (どれかでＯＫ)
1. セルに表示される「RESTART RUNTIME」ボタンをクリック
2. メニューバーのランタイム -> ランタイムを再起動をクリック

# 正しくインストールできているか、モジュールをインポートして確認 (エラーが表示されなければOK)

In [ ]:
# detectron2のインポートとバージョン確認
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# 本プログラムで使用するライブラリのインポート
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

from detectron2.data.datasets import register_coco_instances

from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
import os
import glob

# 骨格検知の推論を実行

In [ ]:
# 画像が読み込めるかを確認
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('/content/drive/MyDrive/test_image1.jpg') #imgという変数に、画像データを格納
plt.figure(figsize=(20,24))
plt.imshow(img)
plt.show()

In [ ]:
# コンフィグ(初期設定)
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

predictor = DefaultPredictor(cfg) #コンフィグ情報をプレディクタにセット

outputs = predictor(img) # 推論実行

# outputs(推論結果)の中身を可視化
v = Visualizer(img[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# outputsの中身を確認
# 検出した骨格の座標情報が格納することがわかる
print(outputs)

# 別の画像でもう一度骨格推定を実行

In [ ]:
# 画像を読み込み
img2 = cv2.imread('/content/drive/MyDrive/test_image2.jpg')

# 画像を描画
plt.figure(figsize=(20,24))
plt.imshow(img2)
plt.show()

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(img2)
v = Visualizer(img2[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

# インスタンスセグメンテーションの推論実行

In [ ]:
# 設定の初期化
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# 学習済みモデルのロード (今回はResNet50+FPN_3x)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# 閾値の設定
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
# ロードした学習済みモデルをモデルの重みの初期値に設定
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# 上記コンフィグをpredictorという関数に変換
predictor = DefaultPredictor(cfg)

# im(画像)をpredictorに代入して、outputsを出力
outputs = predictor(img)

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

# パノプティックセグメンテーションの推論実行

In [ ]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(img)["panoptic_seg"]
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])